In [1]:
#!/usr/bin/env python3
import sys
import copy
sys.path.append('../../')

from __future__ import print_function
from smodels.base import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.decomposition import decomposer
from smodels.base.physicsUnits import fb, GeV, TeV
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.matching import theoryPrediction
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.matching.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.base.smodelsLogging import setLogLevel, logger
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.tools.printers.masterPrinter import MPrinter
from smodels.tools import ioObjects
import os
from collections import OrderedDict
from configparser import ConfigParser, NoSectionError, NoOptionError
setLogLevel("warning")

In [2]:
# Set the path to the database
database = Database('unittest')
databaseVersion = database.databaseVersion

In [3]:
parameterFile='../testPrinters_parameters.ini'
parser = ConfigParser(inline_comment_prefixes=(';',))
ret = parser.read(parameterFile)
parser.set("printer","outputType","xml")
parser.set("printer","outputFormat","current")

inputFile = '../testFiles/slha/lightEWinos.slha'
outputDir = '../'

In [4]:
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
minmassgap = parser.getfloat("parameters", "minmassgap") * GeV

masterPrinter = MPrinter()
masterPrinter.setPrinterOptions(parser)
masterPrinter.setOutPutFiles(os.path.join(
    outputDir, os.path.basename(inputFile)))
# for printer in masterPrinter.Printers.values():
#     printer.output = 'stdout'


In [5]:
masterPrinter.addObj(database)

inputStatus = ioObjects.FileStatus()
if parser.getboolean("options", "checkInput"):
    inputStatus.checkFile(inputFile)

printParameters = []
if parser.has_section('parameters'):
    printParameters += list(parser.items('parameters'))
if parser.has_section('particles'):
    printParameters += list(parser.items('particles'))
if parser.has_section('options'):
    printParameters += list(parser.items('options'))

printParameters = OrderedDict(printParameters)
outputStatus = ioObjects.OutputStatus(inputStatus.status, inputFile,
                                      printParameters, databaseVersion)
masterPrinter.addObj(outputStatus)

In [6]:
model = Model(BSMparticles=BSMList, SMparticles=SMList)
promptWidth = None
stableWidth = None
if parser.has_option("particles", "promptWidth"):
    promptWidth = parser.getfloat("particles", "promptWidth")*GeV
if parser.has_option("particles", "stableWidth"):
    stableWidth = parser.getfloat("particles", "stableWidth")*GeV
model.updateParticles(inputFile=inputFile,
                      promptWidth=promptWidth, stableWidth=stableWidth)

In [7]:
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
smstoplist = decomposer.decompose(model, sigmacut,
                                  massCompress=parser.getboolean(
                                      "options", "doCompress"),
                                  invisibleCompress=parser.getboolean(
                                      "options", "doInvisible"),
                                  minmassgap=minmassgap)


In [8]:
masterPrinter.addObj(smstoplist)

In [9]:
allPredictions = []
combineResults = False
useBest = True
combineResults = parser.getboolean("options", "combineSRs")
allSRs = parser.getboolean("options", "reportAllSRs")
if allSRs:  # If set print out all SRs and skip combination
    useBest = False
    combineResults = False

allPredictions = theoryPredictionsFor(database,smstoplist,
                                             useBestDataset=useBest, combinedResults=combineResults)

if parser.getboolean("options", "computeStatistics"):
    for theoPred in allPredictions:
        theoPred.computeStatistics()

maxcond = parser.getfloat("parameters", "maxcond")
theoryPredictions = theoryPrediction.TheoryPredictionList(
    allPredictions, maxcond)


In [10]:
outputStatus.updateStatus(1)
masterPrinter.addObj(theoryPredictions)

In [11]:
if parser.getboolean("options", "testCoverage"):
    """ Testing coverage of model point, add results to the output file """
    if parser.has_option("options", "coverageSqrts"):
        sqrts = parser.getfloat("options", "coverageSqrts")*TeV
    else:
        sqrts = None
    uncovered = coverage.Uncovered(
        smstoplist, sigmacut=sigmacut, sqrts=sqrts)
    masterPrinter.addObj(uncovered)


In [12]:
if parser.has_option("options", "combineAnas"):
    """ Combine analyses """

    combineAnas = parser.get("options", "combineAnas").replace(" ","").split(",")
    if combineAnas:
        if combineResults is True:
            logger.warning("Combining analyses with signal region combination (combineSRs=True) might significantly reduce CPU performance.")
        combiner = TheoryPredictionsCombiner.selectResultsFrom(theoryPredictions,
                                                               combineAnas)
        # Only compute combination if at least one result was selected
        if combiner is not None:
            combiner.computeStatistics()
            masterPrinter.addObj(combiner)


In [13]:
def pprint(root,tag=None,depth=None):
    if tag is not None:
        entry = root.find(tag)
    else:
        entry = root
    print(entry.tag)
    if not len(entry):
        try:
            print('%1.2e'%float(entry.text))
        except (ValueError,TypeError):
            print('%s'%(entry.text))
    else:
        if depth is not None:
            xlist = entry[:depth]
        else:
            xlist = entry[:]
        for x in xlist:
            pprint(x,depth=depth)

In [14]:
masterCopy = copy.deepcopy(masterPrinter)
x = masterPrinter.flush()['xml']
os.remove(masterPrinter.Printers['xml'].filename)

In [15]:
sortedTags = sorted([el.tag for el in x])
print(sortedTags)

['CombinedRes_List', 'ExptRes_List', 'OutputStatus', 'SMS_Decomposition_List', 'Total_xsec_for_missing_topologies_fb', 'Total_xsec_for_missing_topologies_with_displaced_decays_fb', 'Total_xsec_for_missing_topologies_with_prompt_decays_fb', 'Total_xsec_for_topologies_outside_the_grid_fb', 'missing_topologies_List', 'missing_topologies_with_displaced_decays_List', 'missing_topologies_with_prompt_decays_List', 'topologies_outside_the_grid_List']


In [16]:
pprint(x,'CombinedRes_List')

CombinedRes_List
CombinedRes
AnalysisID
ATLAS-CONF-2013-037
l_SM
2.07e-02
l_max
2.07e-02
likelihood
1.90e-02
r
1.09e-01
r_expected
1.01e-01


In [17]:
pprint(x,'ExptRes_List',depth=3)

ExptRes_List
ExptRes
AnalysisID
ATLAS-CONF-2013-037
AnalysisSqrts_TeV
8.00e+00
DataSetID
SRbC3
ExptRes
AnalysisID
ATLAS-SUSY-2013-02
AnalysisSqrts_TeV
8.00e+00
DataSetID
None


In [18]:
pprint(x,'OutputStatus')

OutputStatus
checkinput
True
combineanas
ATLAS-CONF-2013-037,CMS-SUS-13-012
combinesrs
False
computestatistics
True
database_version
unittest300-beta
decomposition_status
1.00e+00
docompress
True
doinvisible
True
experimentalfeatures
False
file_status
1.00e+00
input_file
../testFiles/slha/lightEWinos.slha
maxcond
2.00e-01
minmassgap
5.00e+00
model
share.models.mssm
ncpus
1.00e+00
promptwidth
1.00e-11
reportallsrs
False
sigmacut
1.00e+01
smodels_version
3.0.0-beta
stablewidth
1.00e-25
testcoverage
True
warnings
Input file ok


In [19]:
pprint(x,'SMS_Decomposition_List',depth=2)

SMS_Decomposition_List
SMS_Decomposition
ID
1.00e+00
Masses_GeV_List
Masses_GeV
('N1', 68.1)
Masses_GeV
('C1+/C1-', 134.4)
SMS_Decomposition
ID
2.00e+00
Masses_GeV_List
Masses_GeV
('N1', 68.1)
Masses_GeV
('C1+/C1-', 134.4)


In [20]:
keys = ['Total_xsec_for_missing_topologies_fb', 
        'Total_xsec_for_missing_topologies_with_displaced_decays_fb', 
        'Total_xsec_for_missing_topologies_with_prompt_decays_fb', 
        'Total_xsec_for_topologies_outside_the_grid_fb']
for k in keys:
    pprint(x,k)

Total_xsec_for_missing_topologies_fb
5.22e+03
Total_xsec_for_missing_topologies_with_displaced_decays_fb
0.00e+00
Total_xsec_for_missing_topologies_with_prompt_decays_fb
5.22e+03
Total_xsec_for_topologies_outside_the_grid_fb
4.72e+01


In [21]:
pprint(x,'missing_topologies_List',depth=2)

missing_topologies_List
missing_topologies
SMS
PV > (jet,jet,MET), (nu,l,MET)
SMS_IDs_List
SMS_IDs
3.10e+01
SMS_IDs
3.60e+01
missing_topologies
SMS
PV > (jet,jet,MET), (nu,ta,MET)
SMS_IDs_List
SMS_IDs
4.10e+01
SMS_IDs
2.60e+01


In [22]:
pprint(x,'missing_topologies_with_displaced_decays_List')

missing_topologies_with_displaced_decays_List
None


In [23]:
pprint(x,'missing_topologies_with_prompt_decays_List',depth=2)

missing_topologies_with_prompt_decays_List
missing_topologies_with_prompt_decays
SMS
PV > (jet,jet,MET), (nu,l,MET)
SMS_IDs_List
SMS_IDs
3.10e+01
SMS_IDs
3.60e+01
missing_topologies_with_prompt_decays
SMS
PV > (jet,jet,MET), (nu,ta,MET)
SMS_IDs_List
SMS_IDs
4.10e+01
SMS_IDs
2.60e+01


In [24]:
pprint(x,'topologies_outside_the_grid_List',depth=2)

topologies_outside_the_grid_List
topologies_outside_the_grid
SMS
PV > (b,b,MET), (b,b,MET)
SMS_IDs_List
SMS_IDs
6.20e+01


In [25]:
masterCopy.outputFormat = "version2"
for printer in masterCopy.Printers.values():
    printer.outputFormat = "version2"
    
x = masterCopy.flush()['xml']
os.remove(masterPrinter.Printers['xml'].filename)

In [26]:
sortedTags = sorted([el.tag for el in x])
print(sortedTags)

['CombinedRes_List', 'Element_List', 'ExptRes_List', 'OutputStatus', 'Total_xsec_for_missing_topologies_fb', 'Total_xsec_for_missing_topologies_with_displaced_decays_fb', 'Total_xsec_for_missing_topologies_with_prompt_decays_fb', 'Total_xsec_for_topologies_outside_the_grid_fb', 'missing_topologies_List', 'missing_topologies_with_displaced_decays_List', 'missing_topologies_with_prompt_decays_List', 'topologies_outside_the_grid_List']


In [27]:
pprint(x,'CombinedRes_List')

CombinedRes_List
CombinedRes
AnalysisID
ATLAS-CONF-2013-037
l_SM
2.07e-02
l_max
2.07e-02
likelihood
1.90e-02
r
1.09e-01
r_expected
1.01e-01


In [28]:
pprint(x,'ExptRes_List',depth=3)

ExptRes_List
ExptRes
AnalysisID
ATLAS-CONF-2013-037
AnalysisSqrts_TeV
8.00e+00
DataSetID
SRbC3
ExptRes
AnalysisID
ATLAS-SUSY-2013-02
AnalysisSqrts_TeV
8.00e+00
DataSetID
None


In [29]:
pprint(x,'OutputStatus')

OutputStatus
checkinput
True
combineanas
ATLAS-CONF-2013-037,CMS-SUS-13-012
combinesrs
False
computestatistics
True
database_version
unittest300-beta
decomposition_status
1.00e+00
docompress
True
doinvisible
True
experimentalfeatures
False
file_status
1.00e+00
input_file
../testFiles/slha/lightEWinos.slha
maxcond
2.00e-01
minmassgap
5.00e+00
model
share.models.mssm
ncpus
1.00e+00
promptwidth
1.00e-11
reportallsrs
False
sigmacut
1.00e+01
smodels_version
3.0.0-beta
stablewidth
1.00e-25
testcoverage
True
warnings
Input file ok


In [30]:
pprint(x,'Element_List',depth=2)

Element_List
Element
ID
1.00e+00
Masses_GeV_List
Masses_GeV_List
Masses_GeV
6.81e+01
Masses_GeV_List
Masses_GeV
1.34e+02
Masses_GeV
6.81e+01
Element
ID
2.00e+00
Masses_GeV_List
Masses_GeV_List
Masses_GeV
6.81e+01
Masses_GeV_List
Masses_GeV
1.34e+02
Masses_GeV
6.81e+01


In [31]:
keys = ['Total_xsec_for_missing_topologies_fb', 
        'Total_xsec_for_missing_topologies_with_displaced_decays_fb', 
        'Total_xsec_for_missing_topologies_with_prompt_decays_fb', 
        'Total_xsec_for_topologies_outside_the_grid_fb']
for k in keys:
    pprint(x,k)

Total_xsec_for_missing_topologies_fb
5.22e+03
Total_xsec_for_missing_topologies_with_displaced_decays_fb
0.00e+00
Total_xsec_for_missing_topologies_with_prompt_decays_fb
5.22e+03
Total_xsec_for_topologies_outside_the_grid_fb
4.72e+01


In [32]:
pprint(x,'missing_topologies_List',depth=2)

missing_topologies_List
missing_topologies
element
[[[jet,jet]],[[l,nu]]] (MET,MET)
element_IDs_List
element_IDs
3.10e+01
element_IDs
3.60e+01
missing_topologies
element
[[[jet,jet]],[[ta,nu]]] (MET,MET)
element_IDs_List
element_IDs
4.10e+01
element_IDs
2.60e+01


In [33]:
pprint(x,'missing_topologies_with_displaced_decays_List')

missing_topologies_with_displaced_decays_List
None


In [34]:
pprint(x,'missing_topologies_with_prompt_decays_List',depth=2)

missing_topologies_with_prompt_decays_List
missing_topologies_with_prompt_decays
element
[[[jet,jet]],[[l,nu]]] (MET,MET)
element_IDs_List
element_IDs
3.10e+01
element_IDs
3.60e+01
missing_topologies_with_prompt_decays
element
[[[jet,jet]],[[ta,nu]]] (MET,MET)
element_IDs_List
element_IDs
4.10e+01
element_IDs
2.60e+01


In [35]:
pprint(x,'topologies_outside_the_grid_List',depth=2)

topologies_outside_the_grid_List
topologies_outside_the_grid
element
[[[b,b]],[[b,b]]] (MET,MET)
element_IDs_List
element_IDs
6.20e+01
